In [ ]:
import json

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load simulation result

In [ ]:
SIMULATION_RESULT_JSON_PATH = "../output/simulation_result.json"

In [ ]:
from perp_simulation.entity.simulation import Simulation


simulation_result_dict = json.load(open(SIMULATION_RESULT_JSON_PATH))
simulation_result = Simulation.from_dict(simulation_result_dict)

## Simulation info

In [ ]:
print(f"Simulation name: {simulation_result.name}")
print(f"Simulation start time: {simulation_result.simulation_start_ts}")
print(f"Simulation end time: {simulation_result.simulation_end_ts}")
print(f"Simulation symbol: {simulation_result.symbol}")
print(f"Simulation timeframe: {simulation_result.timeframe}")
print(
    f"Simulation duration: {simulation_result.run_end_ts - simulation_result.run_start_ts} seconds"
)

# Load data

In [ ]:
DATA_BASE_PATH = "../data/binance-futures"

In [ ]:
from perp_simulation.gateway.ohlcv_repository import OHLCVRepository


ohlcv_repository = OHLCVRepository(DATA_BASE_PATH)

In [ ]:
from datetime import datetime, timezone

ohlcv_df = ohlcv_repository.get_historical_dataframe(
    simulation_result.symbol,
    datetime.fromtimestamp(simulation_result.simulation_start_ts, tz=timezone.utc),
    simulation_result.timeframe,
)

In [ ]:
ohlcv_df.head()

In [ ]:
ohlcv_df.tail()

# Analysis

Analyze the progression of:
- Account balance
- Account margin balance
- Position notional value
- Position unrealized PnL
- Position effective leverage
- Position liquidation price

In [ ]:
from typing import List
from perp_simulation.entity.account_snapshot import AccountSnapshot


def get_account_progression(account_snapshots: List[AccountSnapshot]) -> pd.DataFrame:
    snapshot_tss = []
    account_balances = []
    position_quantities = []
    position_unrealized_pnls = []
    position_effective_leverages = []
    position_liquidation_prices = []
    for snapshot in account_snapshots:
        snapshot_tss.append(snapshot.ts)
        account_balances.append(snapshot.account.balance)
        if snapshot.account.positions is None or len(snapshot.account.positions) == 0:
            position_quantities.append(None)
            position_unrealized_pnls.append(None)
            position_effective_leverages.append(None)
            position_liquidation_prices.append(None)
        else:
            position_quantities.append(snapshot.account.positions[0].quantity)
            position_unrealized_pnls.append(
                snapshot.account.positions[0].unrealized_pnl
            )
            position_effective_leverages.append(
                snapshot.account.positions[0].effective_leverage
            )
            position_liquidation_prices.append(
                snapshot.account.positions[0].liquidation_price
            )
    _index = pd.to_datetime(snapshot_tss, unit="s", utc=True)
    _df = pd.DataFrame(
        {
            "account_balance": account_balances,
            "position_quantity": position_quantities,
            "position_unrealized_pnl": position_unrealized_pnls,
            "position_effective_leverage": position_effective_leverages,
            "position_liquidation_price": position_liquidation_prices,
        },
        index=_index,
    )
    return _df

In [ ]:
progression_df = get_account_progression(simulation_result.account_snapshots)

In [ ]:
progression_df.head()

In [ ]:
progression_df.tail()

In [ ]:
def plot_account_with_price(df: pd.DataFrame, price_df: pd.DataFrame):
    fig = px.line(
        df, x=df.index, y="account_balance", title="Account balance progression"
    )
    fig.add_scatter(x=price_df.index, y=price_df["close"], mode="lines", name="Price")
    fig.show()

In [ ]:
to_str = "2024-01-24"
plot_account_with_price(progression_df.loc[:to_str, :], ohlcv_df.loc[:to_str, :])

In [ ]:
def plot_position_with_price(df: pd.DataFrame, price_df: pd.DataFrame):
    # Calculate values
    position_notional_value = df["position_quantity"] * price_df["close"]
    position_unrealized_pnl = df["position_unrealized_pnl"]
    position_effective_leverage = df["position_effective_leverage"]
    position_liquidation_price = df["position_liquidation_price"]

    # Create a figure
    fig = go.Figure()

    # Add traces
    fig.add_trace(
        go.Scatter(
            x=position_notional_value.index,
            y=position_notional_value,
            mode="lines",
            name="Position notional value",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=position_unrealized_pnl.index,
            y=position_unrealized_pnl,
            mode="lines",
            name="Position unrealized PnL",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=position_effective_leverage.index,
            y=position_effective_leverage,
            mode="lines",
            name="Position effective leverage",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=position_liquidation_price.index,
            y=position_liquidation_price,
            mode="lines",
            name="Position liquidation price",
        )
    )
    fig.add_trace(
        go.Scatter(x=price_df.index, y=price_df["close"], mode="lines", name="Price")
    )

    # Show the figure
    fig.show()

In [ ]:
plot_position_with_price(progression_df.loc[:to_str, :], ohlcv_df.loc[:to_str, :])